<a href="https://colab.research.google.com/github/Ashish-Soni08/Playground/blob/main/LlamaIndex/Metadata_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting Metadata for Better Document Indexing and Understanding

In many cases, especially with long documents, a chunk of text may lack the context necessary to disambiguate the chunk from other similar chunks of text. One method of addressing this is manually labelling each chunk in our dataset or knowledge base. However, this can be labour intensive and time consuming for a large number or continually updated set of documents.

To combat this, we use LLMs to extract certain contextual information relevant to the document to better help the retrieval and language models disambiguate similar-looking passages.

We do this through our brand-new `MetadataExtractor` modules.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [1]:
%%capture

!pip install llama-index pypdf

In [2]:
import nest_asyncio

nest_asyncio.apply()

import os
import openai

openai.api_key = '' # OPENAI_API_KEY

In [3]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.schema import MetadataMode

In [4]:
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo", max_tokens=512)

We create a node parser that extracts the document title and hypothetical question embeddings relevant to the document chunk.

We also show how to instantiate the `SummaryExtractor` and `KeywordExtractor`, as well as how to create your own custom extractor
based on the `MetadataFeatureExtractor` base class

In [5]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.node_parser.extractors import (
    MetadataExtractor,
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    EntityExtractor,
    MetadataFeatureExtractor,
)
from llama_index.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(
    separator=" ", chunk_size=512, chunk_overlap=128
)


class CustomExtractor(MetadataFeatureExtractor):
    def extract(self, nodes):
        metadata_list = [
            {
                "custom": (
                    node.metadata["document_title"]
                    + "\n"
                    + node.metadata["excerpt_keywords"]
                )
            }
            for node in nodes
        ]
        return metadata_list


metadata_extractor = MetadataExtractor(
    extractors=[
        TitleExtractor(nodes=5, llm=llm),
        QuestionsAnsweredExtractor(questions=3, llm=llm),
    ],
)

node_parser = SimpleNodeParser.from_defaults(
    text_splitter=text_splitter,
    metadata_extractor=metadata_extractor,
)

In [6]:
from llama_index import SimpleDirectoryReader

We first load the 10k annual SEC report for Uber and Lyft for the years 2019 and 2020 respectively.

In [7]:
!mkdir -p data
!wget -O "data/10k-132.pdf" "https://www.dropbox.com/scl/fi/6dlqdk6e2k1mjhi8dee5j/uber.pdf?rlkey=2jyoe49bg2vwdlz30l76czq6g&dl=1"
!wget -O "data/10k-vFinal.pdf" "https://www.dropbox.com/scl/fi/qn7g3vrk5mqb18ko4e5in/lyft.pdf?rlkey=j6jxtjwo8zbstdo4wz3ns8zoj&dl=1"

--2023-10-28 07:31:02--  https://www.dropbox.com/scl/fi/6dlqdk6e2k1mjhi8dee5j/uber.pdf?rlkey=2jyoe49bg2vwdlz30l76czq6g&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc836d4a9337a31f04e209287e62.dl.dropboxusercontent.com/cd/0/inline/CGfK7z3X33ouk78YCYBCyMcwauuLu_edKu7x9UqQKItPT5JpM8k8gslLbZLybXVhETNXNp_mL_A4d44_BeXWWjbKxmJZYbclwiXXr8IiD4IrD23aUyvdDthIDtMVC66-mQ4/file?dl=1# [following]
--2023-10-28 07:31:03--  https://uc836d4a9337a31f04e209287e62.dl.dropboxusercontent.com/cd/0/inline/CGfK7z3X33ouk78YCYBCyMcwauuLu_edKu7x9UqQKItPT5JpM8k8gslLbZLybXVhETNXNp_mL_A4d44_BeXWWjbKxmJZYbclwiXXr8IiD4IrD23aUyvdDthIDtMVC66-mQ4/file?dl=1
Resolving uc836d4a9337a31f04e209287e62.dl.dropboxusercontent.com (uc836d4a9337a31f04e209287e62.dl.dropboxusercontent.com)... 162.125.65.15, 2620:100:6021:15::a27d:410f
Connect

In [8]:
# Note the uninformative document file name, which may be a common scenario in a production setting
uber_docs = SimpleDirectoryReader(input_files=["data/10k-132.pdf"]).load_data()
uber_front_pages = uber_docs[0:3]
uber_content = uber_docs[63:69]
uber_docs = uber_front_pages + uber_content

In [9]:
uber_nodes = node_parser.get_nodes_from_documents(uber_docs)

Extracting questions:   0%|          | 0/21 [00:00<?, ?it/s]

In [10]:
uber_nodes[6].metadata

{'page_label': '64',
 'file_name': '10k-132.pdf',
 'document_title': 'Exploring the Diverse Landscape of 2019: A Comprehensive Annual Report on Uber Technologies, Inc.',
 'questions_this_excerpt_can_answer': '1. How does Uber measure the adoption and frequency of transactions on its platform?\n2. What is the definition of "Trips" according to Uber and how does it help measure the scale and usage of their platform?\n3. How does Uber count unique consumers and why is it important for assessing the penetration of their services in different countries?'}

In [11]:
# Note the uninformative document file name, which may be a common scenario in a production setting
lyft_docs = SimpleDirectoryReader(
    input_files=["data/10k-vFinal.pdf"]
).load_data()
lyft_front_pages = lyft_docs[0:3]
lyft_content = lyft_docs[68:73]
lyft_docs = lyft_front_pages + lyft_content

In [12]:
lyft_nodes = node_parser.get_nodes_from_documents(lyft_docs)

Extracting questions:   0%|          | 0/19 [00:00<?, ?it/s]

In [13]:
lyft_nodes[2].metadata

{'page_label': '2',
 'file_name': '10k-vFinal.pdf',
 'document_title': 'Lyft, Inc. Annual Report on Form 10-K for the fiscal year ended December 31, 2020',
 'questions_this_excerpt_can_answer': "1. What is the aggregate market value of Lyft, Inc.'s common stock held by non-affiliates as of June 30, 2020?\n2. How many shares of Class A common stock and Class B common stock does Lyft, Inc. have outstanding as of February 22, 2021?\n3. Has Lyft, Inc. filed a report on and attestation to its management's assessment of the effectiveness of its internal control over financial reporting under Section 404(b) of the Sarbanes-Oxley Act?"}

Since we are asking fairly sophisticated questions, we utilize a subquestion query engine for all QnA pipelines below, and prompt it to pay more attention to the relevance of the retrieved sources.

## Querying an Index With No Extra Metadata

In [14]:
from copy import deepcopy

nodes_no_metadata = deepcopy(uber_nodes) + deepcopy(lyft_nodes)
for node in nodes_no_metadata:
    node.metadata = {
        k: node.metadata[k]
        for k in node.metadata
        if k in ["page_label", "file_name"]
    }
print(
    "LLM sees:\n",
    (nodes_no_metadata)[9].get_content(metadata_mode=MetadataMode.ALL),
)

LLM sees:
 [Excerpt from document]
page_label: 65
file_name: 10k-132.pdf
Excerpt:
-----
income (loss) attributable to  Uber Technologies, Inc. to Adjusted EBITDA. 
            
  Year Ended December 31,   2017 to 2018   2018 to 2019   
(In millions, exce pt percenta ges)  2017   2018   2019   % Chan ge  % Chan ge  
Adjusted EBITDA ................................  $ (2,642) $ (1,847) $ (2,725)  30%  (48)%
-----


In [15]:
from llama_index import VectorStoreIndex
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.tools import QueryEngineTool, ToolMetadata

In [32]:
index_no_metadata = VectorStoreIndex(
    nodes=nodes_no_metadata,
    service_context=ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo-0613")),
)
engine_no_metadata = index_no_metadata.as_query_engine(
    similarity_top_k=10,
)

In [33]:
final_engine_no_metadata = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[
        QueryEngineTool(
            query_engine=engine_no_metadata,
            metadata=ToolMetadata(
                name="sec_filing_documents",
                description="financial information on companies",
            ),
        )
    ],
    use_async=True,
)

In [34]:
response_no_metadata = final_engine_no_metadata.query(
    """
    What was the cost due to research and development v.s. sales and marketing for uber and lyft in 2019 in millions of USD?
    Give your answer as a JSON.
    """
)
# {"Uber": {"Research and Development": 4836, "Sales and Marketing": 4626},
#  "Lyft": {"Research and Development": 1505.6, "Sales and Marketing": 814 }}

Generated 4 sub questions.
[sec_filing_documents] Q: What was the cost due to research and development for Uber in 2019 in millions of USD?
[sec_filing_documents] Q: What was the cost due to sales and marketing for Uber in 2019 in millions of USD?
[sec_filing_documents] Q: What was the cost due to research and development for Lyft in 2019 in millions of USD?
[sec_filing_documents] Q: What was the cost due to sales and marketing for Lyft in 2019 in millions of USD?
[sec_filing_documents] A: The cost due to sales and marketing for Uber in 2019 was $814.1 million.
[sec_filing_documents] A: The cost due to research and development for Uber in 2019 was $1,505.6 million.
[sec_filing_documents] A: The cost due to sales and marketing for Lyft in 2019 was $814.1 million.
[sec_filing_documents] A: The cost due to research and development for Lyft in 2019 was $276.6 million.


In [35]:
print(response_no_metadata.response)

{
  "Uber": {
    "Research and Development": 1505.6,
    "Sales and Marketing": 814.1
  },
  "Lyft": {
    "Research and Development": 276.6,
    "Sales and Marketing": 814.1
  }
}


**RESULT**: As we can see, the QnA agent does not seem to know where to look for the right documents. As a result it gets the Lyft and Uber data completely mixed up.

## Querying an Index With Extracted Metadata

In [24]:
print(
    "LLM sees:\n",
    (uber_nodes + lyft_nodes)[9].get_content(metadata_mode=MetadataMode.ALL),
)

LLM sees:
 [Excerpt from document]
page_label: 65
file_name: 10k-132.pdf
document_title: Exploring the Diverse Landscape of 2019: A Comprehensive Annual Report on Uber Technologies, Inc.
questions_this_excerpt_can_answer: 1. What was the percentage change in Adjusted EBITDA for Uber Technologies, Inc. from 2017 to 2018 and from 2018 to 2019?
2. What was the income (loss) attributable to Uber Technologies, Inc. in millions for the years 2017, 2018, and 2019?
3. How does the Adjusted EBITDA for Uber Technologies, Inc. in 2019 compare to the previous two years in terms of percentage change?
Excerpt:
-----
income (loss) attributable to  Uber Technologies, Inc. to Adjusted EBITDA. 
            
  Year Ended December 31,   2017 to 2018   2018 to 2019   
(In millions, exce pt percenta ges)  2017   2018   2019   % Chan ge  % Chan ge  
Adjusted EBITDA ................................  $ (2,642) $ (1,847) $ (2,725)  30%  (48)%
-----


In [36]:
index = VectorStoreIndex(
    nodes=uber_nodes + lyft_nodes,
    service_context=ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo-0613")),
)
engine = index.as_query_engine(
    similarity_top_k=10,
)

In [37]:
final_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[
        QueryEngineTool(
            query_engine=engine,
            metadata=ToolMetadata(
                name="sec_filing_documents",
                description="financial information on companies.",
            ),
        )
    ],
    use_async=True,
)

In [38]:
response = final_engine.query(
    """
    What was the cost due to research and development v.s. sales and marketing for uber and lyft in 2019 in millions of USD?
    Give your answer as a JSON.
    """
)
# Correct answer:
# {"Uber": {"Research and Development": 4836, "Sales and Marketing": 4626},
#  "Lyft": {"Research and Development": 1505.6, "Sales and Marketing": 814 }}

Generated 4 sub questions.
[sec_filing_documents] Q: What was the cost due to research and development for Uber in 2019 in millions of USD?
[sec_filing_documents] Q: What was the cost due to sales and marketing for Uber in 2019 in millions of USD?
[sec_filing_documents] Q: What was the cost due to research and development for Lyft in 2019 in millions of USD?
[sec_filing_documents] Q: What was the cost due to sales and marketing for Lyft in 2019 in millions of USD?
[sec_filing_documents] A: The cost due to research and development for Uber in 2019 was not provided in the given context.
[sec_filing_documents] A: The cost due to sales and marketing for Lyft in 2019 was $4,626 million.
[sec_filing_documents] A: The cost due to sales and marketing for Uber in 2019 was $7,208 million.
[sec_filing_documents] A: The cost due to research and development for Lyft in 2019 was $1,505.6 million.


In [39]:
print(response.response)

{
  "Uber": {
    "Research and Development": 0,
    "Sales and Marketing": 7208
  },
  "Lyft": {
    "Research and Development": 1505.6,
    "Sales and Marketing": 4626
  }
}


**RESULT**: As we can see, the LLM answers the questions correctly. (not with GPT-3.5-turbo)